<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/ner-fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import numpy as np

In [10]:
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt'
!mkdir data && mv train.txt data && mv test.txt data

train.txt           100%[===================>]   2.22M  --.-KB/s    in 0.06s   
test.txt            100%[===================>] 245.46K  --.-KB/s    in 0.02s   
mkdir: cannot create directory ‘data’: File exists


# Benchmark2

In [3]:
!cat data/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > train.txt.tmp
!cat data/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > test.txt.tmp

In [4]:
!git clone https://github.com/huggingface/transformers
!pip install transformers/
!pip install sentencepiece

Cloning into 'transformers'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 60794 (delta 27), reused 49 (delta 15), pack-reused 60723
Receiving objects: 100% (60794/60794), 45.62 MiB | 24.96 MiB/s, done.
Resolving deltas: 100% (42914/42914), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 8.1MB/s 
     |████████████████████████████████| 2.9MB 7.0MB/s 
  Created wheel for transformers: filename=transformers-4.3.0.dev0-cp36-none-any.whl size=1783249 sha256=bbe4e984b398d3739016ee7295c04953dc0c0609fa9c7c181a5b0330c2bd99e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-jizfjd_7/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-

In [5]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-01-27 16:31:22--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-01-27 16:31:22 (52.7 MB/s) - ‘preprocess.py’ saved [991/991]



In [6]:
# !python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > train.txt
!python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > train.txt
!python3 preprocess.py test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > test.txt

Downloading: 100% 434/434 [00:00<00:00, 298kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 3.16MB/s]


In [7]:
!pip install -r transformers/examples/token-classification/requirements.txt

     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 163kB 12.2MB/s 
     |████████████████████████████████| 20.7MB 7.5MB/s 
     |████████████████████████████████| 245kB 43.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=3ceecc13602768847ae6d21713e844b94ffa7025a38179c259963ca0a665e214
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [8]:
!mkdir processed_data
!mv train.txt processed_data/ && mv test.txt processed_data/

In [9]:
!cat processed_data/train.txt processed_data/test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [ ]:
!python3 transformers/examples/legacy/token-classification/run_ner.py --data_dir data/ \
--labels labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length  128 \
--save_steps 750 \
--seed 1 \
--do_eval

In [ ]:
!pip install conllu

In [ ]:
import pandas as pd

In [ ]:
with open('data-fa/train.csv', 'r+') as file:
  s = ''
  for line in file.readlines():
    s += line.replace(',', '\t')
  file.seek = 0
  file.write(s)

In [ ]:
!cat data-fa/train.csv

In [ ]:
!curl -L 'https://drive.google.com/file/d/1Jjhbal535VVz2ap4v4r_rN1UEHTdLK5P/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > train.txt.tmp
!curl -L 'https://drive.google.com/file/d/1ZfRcQThdtAR5PPRjIDtrVP7BtXSCUBbm/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > dev.txt.tmp
!curl -L 'https://drive.google.com/file/d/1u9mb7kNJHWQCWyweMDRMuTFoOHOfeBTH/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > test.txt.tmp